In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.whosampled

import pandas as pd

df = pd.DataFrame(list(db.main_redo.find()))

In [9]:
#Turn sampled_song_producer from list
df['sampled_song_producer'] = df.sampled_song_producer.apply( lambda x: ''.join(x))

df.drop(columns= "_id", inplace=True)

#Only 300 or so dupes
df.drop_duplicates(inplace=True)

In [16]:
df.head()

,URL,contributor_points,elements_sampled,name_of_contributor,new_song_album,new_song_artist,new_song_name,new_song_producer,new_song_year,"presence_of_""and throughout""_in_description",sampled_artist,sampled_song_album,sampled_song_name,sampled_song_producer,sampled_song_year,time_in_sampled_song_where_sample_appears
0,https://www.whosampled.com/sample/126703/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Hook / Riff,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),The 45 King,1988,Sample appears at 1:24 (and throughout),James Brown,It's a New Day - Let a Man Come In,It's a New Day,James Brown,1970,0:12
1,https://www.whosampled.com/sample/126703/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Hook / Riff,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),Marley Marl,1988,Sample appears at 1:24 (and throughout),James Brown,It's a New Day - Let a Man Come In,It's a New Day,James Brown,1970,0:12
2,https://www.whosampled.com/sample/126703/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Hook / Riff,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),Marley Marl,1988,Sample appears at 1:24 (and throughout),James Brown,It's a New Day - Let a Man Come In,It's a New Day,James Brown,1970,0:12
3,https://www.whosampled.com/sample/126696/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Vocals / Lyrics,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),The 45 King,1988,Sample appears at 0:00,Lyn Collins,None Listed,"We Want to Parrty, Parrty, Parrty",James Brown,1973,0:00
4,https://www.whosampled.com/sample/126696/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Vocals / Lyrics,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),Marley Marl,1988,Sample appears at 0:00,Lyn Collins,None Listed,"We Want to Parrty, Parrty, Parrty",James Brown,1973,0:00


In [5]:
type(df)

pandas.core.frame.DataFrame

In [2]:
df.shape

(16704, 17)